In [1]:
import requests
from bs4 import BeautifulSoup
import random
import re
import json
import copy
from collections import defaultdict
import time
from tqdm import tqdm
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
#nltk.download('punkt')
#nltk.download('stopwords')


In [2]:
####WIKIPEDIA SCRAPING#####
response = requests.get('https://en.wikipedia.org/wiki/Machine_learning')
html = response.text
soup = BeautifulSoup(html, 'html.parser')
stop_words = set(stopwords.words('english'))

# get random wikipedia corpus
alllinks = []
num_links = 0
max_links = 10
print("Scraping links...")
pbar = tqdm(total = max_links)
while(num_links < max_links):
    links = []
    # Get all the links
    for link in soup.findAll('a', attrs={'href': re.compile("^/wiki/")}):
        links.append(link.get('href'))
    random.shuffle(links)
    alllinks.append(links[0])
    soup = BeautifulSoup(requests.get(
        "http://en.wikipedia.org" + links[0]).text, 'html.parser')
    num_links += 1
    pbar.update(1)
pbar.close()
alltext = ""

print("Extracting text...")
# put all words into text file
for link in tqdm(alllinks):
    response = requests.get("http://en.wikipedia.org" + link)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    # extract paragraphs
    text_container = soup.find('div', {'class': 'mw-parser-output'})
    zero_paragaph = {"title": "", "text": ""}
    
    if(text_container == None):
        continue
    current_paragraph = copy.deepcopy(zero_paragaph)
    page = {'paragraphs': []}
    for child in text_container.children:
        if child.name == "p":
            current_paragraph["text"] += child.text + "\n"
        elif child.name == "h2":
            page['paragraphs'].append(current_paragraph)
            current_paragraph = copy.deepcopy(zero_paragaph)
            current_paragraph["title"] = next(child.children).text

        page['paragraphs'] = list(
            filter(lambda x: x["text"] != "", page['paragraphs']))
        
    for pg in page['paragraphs']:
        alltext += pg['text']
        
uncleaned_tokens = word_tokenize(alltext)


print("Cleaning...")
#clean up tokens
stop_words = set(stopwords.words('english'))
tokens = []
for tok in tqdm(uncleaned_tokens):
    tok = tok.lower()
    if(tok.isalnum() and not tok.isdigit()):
        tokens.append(tok)
        
        


Scraping links...


100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


Extracting text...


100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


Cleaning...


100%|██████████| 18986/18986 [00:00<00:00, 1713939.47it/s]


In [3]:
#### CORPUS ####
class Corpus(torch.utils.data.Dataset):
    def __init__(self, tokens, ngrams=1):
        self.ngrams = ngrams
        self.tokens = tokens  #list of words
        self.totlen = len(tokens)
        
        # vocab will be defined over kmers
        self.create_vocab()
        
        
    def create_vocab(self):
        self.vocab_freq = defaultdict(float)
        st_time = time.time()
        for word in self.tokens:
            # compute freq of each word
            if word in self.vocab_freq:
                self.vocab_freq[word] += 1
            else:
                self.vocab_freq[word] = 1
     
        
        self.vocab = sorted(self.vocab_freq.keys())
        
        # create forward and reverse index for all the words in vocab
        self.word_to_idx = defaultdict(lambda: 0)
        self.idx_to_word = defaultdict(lambda: 0)
        for idx, w in enumerate(self.vocab):
            self.word_to_idx[w] = idx
            self.idx_to_word[idx] = w
        
        self.vocab_prob = np.array([self.vocab_freq[k] for k in self.vocab])
        self.vocab_freq_scaler = 0.73
        total_freq = float(self.vocab_prob.sum())
        self.vocab_prob = self.vocab_prob / total_freq
        self.vocab_csum = np.cumsum(self.vocab_prob)
        
        en_time = time.time()
        print("corpus construct time (seconds):", en_time - st_time, "num tokens:", total_freq)
        
    def __len__(self):
        return self.totlen
    
    
    def __getitem__(self, index):
        if(index >= len(self.tokens) - 15):
            return (
                torch.tensor([self.word_to_idx[word] for word in self.tokens[index:index+10]]),
                torch.tensor([self.word_to_idx[word] for word in self.tokens[index:index+10]]),
            )
        else:
            return (
                torch.tensor([self.word_to_idx[word] for word in self.tokens[index:index+10]]),
                torch.tensor([self.word_to_idx[word] for word in self.tokens[index+1:index+11]]),
            )




In [4]:
#### POS/NEG TARGET CONTEXT CREATION ####
class NegSampler:
    '''generate a block of negative samples from the cumsum array (Cumulative Distribution Function)'''
    def __init__(self, csum_ary):
        self.csum_ary = csum_ary
        self.time = 0.

    def get_neg_words(self, num_words):
        '''get num_words negative words sampled from cumsum array'''
        st_time = time.time()
        nprobs = np.random.random(num_words)
        neg_words = np.searchsorted(self.csum_ary, nprobs)
        en_time = time.time()
        self.time += en_time-st_time
        return neg_words

class PosNegSampler(torch.utils.data.IterableDataset):
    '''This class creates a block of positive and negative pairs for word2vec training
    The iterable will return a numpy array of target, context and label triples'''
    
    def __init__(self, C, window_size, neg_samples, block_size):
        super(PosNegSampler, self).__init__()
        self.window_size = window_size
        self.neg_samples = neg_samples
        self.time = 0.
        self.block_sz = block_size
        self.C = C
        
        self.negsampler = NegSampler(self.C.vocab_csum)
                    
    def context_data(self, block_sz):
        '''generate center word, context word pairs
        '''
        T = []
        C = []
        for i, word in enumerate(self.C.tokens):
            #get window of words and create target context pairs
            start_idx = max(0, i - self.window_size)
            end_idx = min(len(self.C.vocab), i + self.window_size + 1)
            for j in range(start_idx, end_idx):
                if i != j:
                    T.append(self.C.word_to_idx[word])
                    C.append(self.C.word_to_idx[self.C.tokens[j]])

            # return a block of T, C
            if len(T) >= block_sz:
                yield (T, C)
                T, C = [], []
                        
        # return any remining elements           
        yield (T, C)
            

    def __iter__(self):
        '''return one pos word and neq_samples neg words and the labels
           use context_data to retrieve a block
        '''

        st_time = time.time()
        
        for i, (T, C) in enumerate(self.context_data(self.block_sz)):
            Tnp = np.array(T)
            Cnp = np.array(C)
            L = np.ones(len(T))
            yield (Tnp, Cnp, L)
            
            L = np.zeros(len(T))
            N = self.negsampler.get_neg_words(
                len(T))
            Nnp = np.array(N)
            yield (Tnp, Nnp, L)
                
        en_time = time.time()
        self.time += en_time-st_time

In [5]:
class Word2Vec(nn.Module):
    '''The word2vec model to train the word embeddings'''
    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embedding_size = embedding_size
        self.T = nn.Embedding(vocab_size, embedding_size)
        self.C = nn.Embedding(vocab_size, embedding_size)

    def forward(self, target_word, context_word, label):
        t = self.T(target_word)
        c = self.C(context_word)
        out = torch.sum(t * c, dim=1)
        return out
    
    def save_embeddings(self, file_name, idx_to_word):
        # average the T and C matrices
        W = (net.T.weight.cpu().data.numpy() + net.C.weight.cpu().data.numpy())/2.

        with open(file_name, "w") as f:
            f.write("%d %d\n" % (len(idx_to_word), self.embedding_size))
            for wid, w in idx_to_word.items():
                e = ' '.join(map(lambda x: str(x), W[wid]))
                f.write("%s %s\n" % (w, e))


In [6]:
if torch.cuda.is_available():
    device = "cuda"
    print("using device", torch.cuda.get_device_name(device))
else:
    device = "cpu"
    
#scrape and read
c = Corpus(tokens)
window_size = 30
neg_samples = 60
block_size = 1024

PNS = PosNegSampler(c, window_size, neg_samples, block_size)
V = len(PNS.C.vocab)
print("vocab, device: ", V, device)

training_generator = data.DataLoader(
    PNS, batch_size = 1
)


#model parameters
word_embedding_dim = 200
epochs = 200
learning_rate = 0.01


# create the NN model
net = Word2Vec(embedding_size=word_embedding_dim, vocab_size=V)
net.to(device)
net.train()
net
loss_function = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(net.parameters(), lr=learning_rate)


start_t = time.time()
for e in range(epochs):
    running_loss = 0
    for bidx, (targets, contexts, labels) in enumerate(
        tqdm(training_generator)):
        
        targets = targets.flatten().to(device)
        contexts = contexts.flatten().to(device)
        labels = labels.flatten().to(device)
        
        net.zero_grad()
        preds = net(targets, contexts, labels)
        loss = loss_function(preds, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("epoch", e, running_loss, bidx, running_loss / (bidx + 1))
    
end_t = time.time()
print("finished in time", end_t - start_t)


for mode in ['avg', 'target', 'context']:
    output_file = "./word_embeds.vec"
    net.save_embeddings(output_file, PNS.C.idx_to_word)

using device Tesla T4
corpus construct time (seconds): 0.005596637725830078 num tokens: 15322.0
vocab, device:  2795 cuda


310it [00:00, 639.99it/s]


epoch 0 1276.6990155759395 309 4.118383921212708


310it [00:00, 684.67it/s]


epoch 1 712.2459422808568 309 2.297567555744699


...

epoch 197 128.57573419373028 309 0.4147604328830009


310it [00:00, 714.57it/s]


epoch 198 129.24442992398352 309 0.4169175158838178


310it [00:00, 711.43it/s]


epoch 199 128.12086866843256 309 0.41329312473687924
finished in time 87.0692389011383


In [ ]:
class TextGenerate(nn.Module):
    def __init__(self, C, word_embed_file, lstm_dim, embedding_dim, num_layers):
        super(TextGenerate, self).__init__()
        self.lstm_dim = lstm_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        n_vocab = len(C)
        
        embeddings_index = {}
        with open(word_embed_file) as f:
            for line in f: 
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
        embeddings_matrix = np.zeros((n_vocab, embedding_dim))
        for word, i in C.word_to_idx.items():
            embedding_vector = embeddings_index.get(word)
            if(embedding_vector is not None):
                embeddings_matrix[i] = embedding_vector
        print(embeddings_matrix.shape)
        
        self.embedding = nn.Embedding(num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim, _weight=torch.Tensor(embeddings_matrix))
        self.embedding.requires_grad = False
        self.lstm = nn.LSTM(input_size = self.lstm_dim, hidden_size = self.lstm_dim, num_layers = self.num_layers, dropout=0.2)
        self.fc = nn.Linear(self.lstm_dim, n_vocab)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_vocab, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        logits = self.relu(logits)
        logits = self.fc2(logits)
        return logits, state
        
    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_dim), torch.zeros(self.num_layers, sequence_length, self.lstm_dim))
        


training_generator = data.DataLoader(
    c, batch_size = 1
)





#model parameters
word_embedding_dim = 200
epochs = 5
learning_rate = 0.01


# create the NN model
net2 = TextGenerate(c, './word_embeds.vec', word_embedding_dim, word_embedding_dim, 6)
net2.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net2.parameters(), lr=learning_rate)
net2.train()
net2

start_t = time.time()
for e in range(epochs):
    running_loss = 0
    state_h, state_c = net2.init_state(10)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for bidx, (x, y) in enumerate(tqdm(training_generator)):
        net2.zero_grad()
        x = x.to(device)
        y = y.to(device)
        if(x.shape[1] != 10 or y.shape[1] != 10):
            continue
        y_pred, (state_h, state_c) = net2(x, (state_h, state_c))
        loss = loss_function(y_pred.transpose(1, 2), y)
        
        state_h = state_h.detach()
        state_c = state_c.detach()
            
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("epoch", e, running_loss, bidx, running_loss / (bidx + 1))
    
end_t = time.time()
print("finished in time", end_t - start_t)
      

(15322, 200)


 22%|██▏       | 3349/15322 [06:24<22:57,  8.69it/s]

In [ ]:
def getResponse(question, num_words_response):
    text = question
    next_words = num_words_response
    words = text.split(' ')
    output = []
    state_h, state_c = net2.init_state(len(words))
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    last_word = ''
    with torch.no_grad():
        for i in range(0, next_words):
            x = torch.tensor([[c.word_to_idx[w] for w in words[i:]]])
            x = x.to(device)
            y_pred, (state_h, state_c) = net2(x, (state_h, state_c))

            last_word_logits = y_pred[0][-1]
            p = torch.nn.functional.softmax(last_word_logits, dim=0).cpu().numpy()
            word_index = np.random.choice(len(last_word_logits), p=p)
            last_word = c.idx_to_word[word_index]
            while(c.idx_to_word[word_index] == last_word):
                word_index = np.random.choice(len(last_word_logits), p=p)
                new_last_word = c.idx_to_word[word_index]
                new_last_word = str(new_last_word)
                if(len(new_last_word) == 1 and new_last_word != 'a'):
                    continue
                if(new_last_word != last_word):
                    last_word = new_last_word
                    break

            words.append(last_word)
            output.append(last_word)
    return output


In [ ]:
#### MACHINE INTERROGATER TURING MACHINE MODEL ####


## GENERATE POS/NEG SAMPLES ##
#POS 1 = Machine generated response
#NEG 0 = Human generated response

class Responses(torch.utils.data.IterableDataset):
    def __init__(self, C, _csv, _chunk_size, pad_sz):
        self.csv = _csv
        self.C = C
        self.chunk_size = _chunk_size
        self.pad_sz = pad_sz
    
    
    def generateMachineResponse(self, question, response_size=15):
        return getResponse(question, response_size)
    
    def __len__(self):
        return 38269/self.chunk_size
    
    def __iter__(self):
        self.bidx = 0
        self.data = pd.read_csv(self.csv, chunksize=self.chunk_size)
        return self
    
    def padout(self, questions, answers):
        
        padded_questions = []
        padded_answers = []
        for question in questions:
            question = np.array(question)
            padright = self.pad_sz - len(question)
            if(padright < 0):
                padded_questions.append(question[0:self.pad_sz])
            else:
                padded_questions.append(np.pad(question, (0, padright), 'constant', constant_values=(0)))
                
        for answer in answers:
            answer = np.array(answer)
            padright = self.pad_sz - len(answer)
            if(padright < 0):
                padded_answers.append(answer[0:self.pad_sz])
            else:
                padded_answers.append(np.pad(answer, (0, padright), 'constant', constant_values=(0)))
                
        return padded_questions, padded_answers
                
        
    def __next__(self):
        chunk_to_process = next(iter(self.data))
        questions = []
        answers = []
        labels = []
        
        #get text
        for question in chunk_to_process['Question'].tolist():
            questions.append([self.C.word_to_idx[x] for x in question.split(' ')])
            
        for answer in chunk_to_process['Answer'].tolist():
            answer = str(answer)
            answers.append([self.C.word_to_idx[x] for x in answer.split(' ')])
            
        labels.append([1 for x in range(self.chunk_size)])
        
        
        for question in chunk_to_process['Question'].tolist():
            questions.append([self.C.word_to_idx[x] for x in question.split(' ')])
            answers.append([self.C.word_to_idx[x] for x in self.generateMachineResponse(question)])
            
        labels.append([0 for x in range(self.chunk_size)])
        
        questions, answers = self.padout(questions, answers)
        
        return torch.cat((torch.Tensor(np.array(questions)), torch.Tensor(np.array(answers))), axis=1), torch.Tensor(labels)
        
        

    





In [ ]:
class MachineInterrogator(nn.Module):
    def __init__(self, indim, hiddim, num_layers, dpout=0.15, bidir=False):
        super(MachineInterrogator, self).__init__()
        self.ind = indim
        self.hid = hiddim
        self.num_layers = num_layers
        self.dropout = dpout
        self.bidir = bidir
        self.lstm = nn.LSTM(self.ind, self.hid, num_layers, dropout=self.dropout, bidirectional=bidir, batch_first=True)
        self.lim = nn.Linear(in_features=self.hid, out_features=1, bias=True)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hid).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hid).to(device)
        x, _= self.lstm(x, (h0, c0))
        x = self.lim(x)
        return x

In [ ]:
r = Responses(c, './half_jokes.csv', 1, 20)
batch_size = 100
dataset_size = 10000
training_generator = data.DataLoader(
    r, batch_size = batch_size
)


#model parameters
input_dim = 40
epochs = 5
learning_rate = 0.01


# create the NN model
net3 = MachineInterrogator(input_dim, 128, 2)
net3 = net3.to(device)

loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net3.parameters(), lr=learning_rate)
net3.train()



In [ ]:
start_t = time.time()
for e in range(epochs):
    correct = 0
    total = 0
    for bidx, (feats, labels) in enumerate(tqdm(training_generator, total=dataset_size//batch_size)):
        net3.zero_grad()
        feats = feats.to(device)
        labels = labels.to(device)
        
        preds = net3(feats)
        loss = loss_function(preds, labels)
        
        preds = torch.round(torch.sigmoid(preds))
        
        loss.backward()
        optimizer.step()

        correct += (preds == labels).sum().item()
        total += len(preds)
        
    print("epoch", e, " acc: ", correct / (total * 2))
    
end_t = time.time()
print("finished in time", end_t - start_t)
      